In [1]:
import tensorflow
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11986879297795574936
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4162846720
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17211705140049518267
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


### LSTM(Long Short Term Memory) Implementation

#### 1. Model and Parameter Setting

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import json

from tqdm import tqdm

In [2]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

#### 2. Train Data

In [3]:
DATA_IN_PATH = './nlp_kor/data_in/'
DATA_OUT_PATH = './nlp_kor/data_out/'
TRAIN_INPUT_DATA = 'nsmc_train_input.npy'
TRAIN_LABEL_DATA = 'nsmc_train_label.npy'
DATA_CONFIGS = 'data_configs.json'

In [4]:
train_input = np.load(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'rb'))
train_label = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'rb'))
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r', encoding='cp949'))

### 3. Define Model Hyperparameter

In [ ]:
model_name = 'rnn_classifier_en'
BATCH_SIZE = 64
NUM_EPOCHS = 5
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

kargs = {'model_name' : model_name,
         'vocab_size' : prepro_configs['vocab_size'],
         'embedding_dimension' : 100,
         'dropout_rate' : 0.2,
         'lstm_dimension' : 150,
         'dense_dimension' : 150,
         'output_dimension' : 1}

#### 4. Model Implementation

In [5]:
class RNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(RNNClassifier, self).__init__(name=kargs['model_name'])
        self.embedding = layers.Embedding(input_dim=kargs['vocab_size'],
                                          ouput_dim=kargs['embedding_dimension'])
        self.lstm_1_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'], return_sequences=True)
        self.lstm_2_layer = tf.keras.layers.LSTM(kargs['lstm_dimension'])
        self.dropout = layers.Dropout(kargs['dropout_rate'])
        self.fc1 = layers.Dense(units=kargs['dense_dimension'],
                                activation=tf.keras.activation.tanh)
        self.fc2 = layers.Dense(units=kargs['output_dimension'],
                                activation=tf.keras.activation.sigmoid)
        
    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.lstm_1_layer(x)
        x = self.lstm_2_layer(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

#### 5. Generate Model

In [6]:
model = RNNClassifier(**kargs)
model.compile(optimizer=tf.keras.optimizer.Adam(1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinayAccuracy(name='accuracy')])

NameError: name 'kargs' is not defined

#### 6. Training the Model

In [ ]:
# EarlyStopping
# overfitting을 막기 위한 earlystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=1)

# monitor='val_accuracy' : 성능 평가 지표로 val_accuracy
# min_delta : val_accuracy 가 낮아져야 성능이 좋아지는 것이지만 낮아지는 폭이 1e-4보다 작아지면 정지
# min_delta : the threashold that triggers the termination (accuracy should at least improve 1e-4)
# patience : no improvement epochs (patience = 1 : 1번 이상 상승이 없으면 종료)

In [ ]:
# ModelCheckpoint
checkpoint_path = DATA_OUT_PATH + model_name + '/weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if not exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=TRUE)
    print("{} -- Folder create complete \n".format(checkpoint_dir))

In [ ]:
cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

In [ ]:
history = model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, 
                    validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

#### 7. Visualize with Graph

In [7]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string], '')
    plt.xlabel('Epochs')
    plt.ylabel(string)
    plt.legend([string, 'val_' + string])
    plt.show()

In [ ]:
plot_graphs(history, 'accuracy')

plot_graphs(history, 'loss')

#### 8. Submit the data

In [ ]:
DATA_OUT_PATH = './data_out/'
TEST_INPUT_DATA = 'test_input.npy'
TEST_ID_DATA = 'test_id.npy'

test_input = np.load(open(DATA_IN_PATH + TEST_INPUT_DATA, 'rb'))

In [ ]:
SAVE_FILE_NM = 'weights.h5'

model.load_weights(os.path.join(DATA_OUT_PATH, model_name, SAVE_FILE_NM))

In [ ]:
predictions = model.predict(test_input, batch_size=BATCH_SIZE)
predictions = predictions.squeeze(-1)

In [ ]:
predictions.shape

In [ ]:
test_id = np.load(open(DATA_IN_PATH + TEST_ID_DATA, 'rb'), allow_pickle=True)

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

output = pd.DataFrame(data={'id':list(test_id), 'sentiment':list(predictions)})
output.to_csv(DATA_OUT_PATH + 'movie_review_result_rnn.csv', index=False, qutoing=3)